#### Investigating how our interchange intervention effect the output.
Through some trials of experiments, we find that for the most of the time, the teacher model will be experience huge output change with the interchange interventions. We use this document to see what interchange intervention is effective (i.e., having salient effects on the outputs).

Imports

In [1]:
import logging
import os
import random
import pickle
import time
import psutil
import math

import numpy as np
import torch
from torch.utils.data import RandomSampler, SequentialSampler
from tqdm import tqdm, trange
import torch
from torch import nn
from torch.optim import AdamW
from torch.utils.data import BatchSampler, DataLoader, RandomSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm

from BERT.pytorch_pretrained_bert.modeling import BertConfig
from BERT.pytorch_pretrained_bert.optimization import BertAdam, warmup_linear
from BERT.pytorch_pretrained_bert.tokenization import BertTokenizer

from distiller import TaskSpecificDistiller
from causal_distiller import TaskSpecificCausalDistiller

from src.argument_parser import default_parser, get_predefine_argv, complete_argument
from src.nli_data_processing import processors, output_modes
from src.data_processing import init_model, get_task_dataloader
from src.modeling import BertForSequenceClassificationEncoder, FCClassifierForSequenceClassification, FullFCClassifierForSequenceClassification
from src.utils import load_model, count_parameters, eval_model_dataloader_nli, eval_model_dataloader
from src.KD_loss import distillation_loss, patience_loss, diito_distillation_loss
from envs import HOME_DATA_FOLDER

logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

Mock Args

In [2]:
#########################################################################
# Prepare Parser
##########################################################################
parser = default_parser()
DEBUG = True
if DEBUG:
    logger.info("IN DEBUG MODE")
    # run simple fune-tuning *teacher* by uncommenting below cmd
    # argv = get_predefine_argv('glue', 'RTE', 'finetune_teacher')

    # run simple fune-tuning *student* by uncommenting below cmd
    # argv = get_predefine_argv('glue', 'RTE', 'finetune_student')

    # run vanilla KD by uncommenting below cmd
    # argv = get_predefine_argv('glue', 'RTE', 'kd')

    # run Patient Teacher by uncommenting below cmd
    argv = get_predefine_argv('glue', 'SST-2', 'kd.cls')
    try:
        args = parser.parse_args(argv)
    except NameError:
        raise ValueError('please uncomment one of option above to start training')
    args.max_training_examples = 1000
else:
    logger.info("IN CMD MODE")
    args = parser.parse_args()
args = complete_argument(args, is_debug=DEBUG)

02/21/2022 13:06:32 - INFO - __main__ -   IN DEBUG MODE
usage: ipykernel_launcher.py [-h] [--task_name TASK_NAME]
                             [--run_name RUN_NAME] [--output_dir OUTPUT_DIR]
                             [--log_every_step LOG_EVERY_STEP]
                             [--log_interval LOG_INTERVAL]
                             [--checkpoint_interval CHECKPOINT_INTERVAL]
                             [--max_seq_length MAX_SEQ_LENGTH]
                             [--max_training_examples MAX_TRAINING_EXAMPLES]
                             [--seed SEED]
                             [--train_batch_size TRAIN_BATCH_SIZE]
                             [--eval_batch_size EVAL_BATCH_SIZE]
                             [--learning_rate LEARNING_RATE]
                             [--num_train_epochs NUM_TRAIN_EPOCHS]
                             [--gradient_accumulation_steps GRADIENT_ACCUMULATION_STEPS]
                             [--fp16 FP16] [--loss_scale LOSS_SCALE]
             

SystemExit: 2

/afs/cs.stanford.edu/u/wuzhengx/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3425: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
